In [19]:
using Zygote

g, _ = gradient((x, y) -> sum(x .* y), [1, 2, 3], [4, 5, 6])

(([4, 5, 6], [1, 2, 3]), getfield(Zygote, Symbol("##66#67")){Zygote.Pullback{Tuple{getfield(Main, Symbol("##21#22")),Array{Int64,1},Array{Int64,1}},Tuple{getfield(Zygote, Symbol("##2070#back#842")){getfield(Zygote, Symbol("##840#841"))},getfield(Zygote, Symbol("##2082#back#846")){getfield(Zygote, Symbol("##844#845")){Base.Broadcast.Broadcasted{Base.Broadcast.DefaultArrayStyle{1},Nothing,typeof(*),Tuple{Array{Int64,1},Array{Int64,1}}},getfield(Zygote, Symbol("#back#834")){Base.Broadcast.Broadcasted{Base.Broadcast.DefaultArrayStyle{1},Tuple{Base.OneTo{Int64}},getfield(Zygote, Symbol("##819#822")){typeof(*)},Tuple{Array{Int64,1},Array{Int64,1}}},Tuple{Array{Int64,1},Array{Int64,1}}}}},getfield(Zygote, Symbol("##1886#back#734")){getfield(Zygote, Symbol("##730#732")){Array{Int64,1}}}}}}(J#getfield(Main, Symbol("##21#22"))(...)))

1. [gradient](https://github.com/FluxML/Zygote.jl/blob/master/src/compiler/interface.jl#L41) in `compiler/interface.jl`
1. [forward](https://github.com/FluxML/Zygote.jl/blob/master/src/compiler/interface.jl#L36) in `compiler/interface.jl`
1. [_forward](https://github.com/FluxML/Zygote.jl/blob/master/src/compiler/interface.jl#L31) in `compiler/interface.jl`
1. [_forward](https://github.com/FluxML/Zygote.jl/blob/master/src/compiler/interface2.jl#L3) in `compiler/interface.jl`.
    - This is a [Generated Function](https://github.com/lcy-seso/dl_framework/blob/master/Julia/notes/Generated_function.ipynb). See the below cell for its implementation.

In [ ]:
@generated function _forward(ctx::Context, f, args...)
  T = Tuple{f,args...}    # in tuple T, f is the type of user-defined forward function
                          # args are types of input arguments of f

  ignore(T) && return :(f(args...), Pullback{$T}(()))
  g = try _lookup_grad(T) catch e e end
  !(g isa Tuple) && return :(f(args...), Pullback{$T}((f,)))
  meta, forw, _ = g
  argnames!(meta, Symbol("#self#"), :ctx, :f, :args)
  forw = varargs!(meta, forw, 3)
  forw = slots!(pis!(inlineable!(forw)))
  return IRTools.update!(meta, forw)
end

Let's dive into the above implementation.

Notes for the input parameters:

1. `f` is the user-defined forward function.
    - **in a generated function, `f` here is the type of of forward functoin**.
    - [**In Julia version 0.5 and later, each function is a singleton instance of a singleton type!**](https://riptutorial.com/julia-lang/example/22879/immutable-types)
        - Like any other singleton value, we can recover the function sin, for example, from typeof(sin).instance.
    - `args` are arguments passed to the forward function.

In [31]:
@show typeof(sin).instance  # X.instance canbe used to access the singleton instance,
typeof(sin).parameters    # See this doc: https://docs.julialang.org/en/v1/devdocs/types/#Tuple-types-1

(typeof(sin)).instance = sin


svec()

Let's see the implementation of `ignore`:
```julia
ignore(T) = all(T -> T <: Type, T.parameters)
```